In [1]:
from __future__ import print_function

import os.path

import pymongo

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
### google mail API

def gmail_authentication():
    SCOPES = ['https://mail.google.com/']
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

In [6]:
service = build('gmail', 'v1', credentials=Credentials.from_authorized_user_file('token.json', ['https://mail.google.com/']))
messages_id = service.users().messages()\
    .list(userId='me', maxResults=10).execute()

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [50]:
### edit form for mongodb
message_result = []
for message in messages_id['messages']:
    msg = service.users().messages().get(userId='me', id=message['id']).execute()
    tmp_dict = {
        'id' : msg.get('id'),
        'threadId' : msg.get('threadId'),
        'labelIds' : msg.get('labelIds'),
        'snippet' : msg.get('snippet')
    }
    headers = msg['payload']['headers']
    for item in headers:
        tmp_dict['headers.'+item['name']] = item['value']
    message_result.append(tmp_dict)    

In [ ]:
from dataclasses import dataclass, field, asdict
from datetime import datetime

### this is for mysql store
def get_time() -> datetime:
    return f"{datetime.now()}"

@dataclass(frozen=True)
class mail_info:
    subject:str = None
    name:str = None
    send_from:str = None
    send_to:str = None
    reply_to:str = None
    date:str = field(default_factory=get_time)

    dict = asdict